In [24]:
import numpy as np
import yfinance as yf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import prophet as pt
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error
sns.set_style('whitegrid')
plt.style.use("default")
%matplotlib inline
from pandas_datareader.data import DataReader
from pandas_datareader import data as pdr
yf.pdr_override()
#from tensorflow import keras
#from keras.layers import Dense
#from keras.models import Sequential, load_model
from sklearn import preprocessing
from sklearn import metrics 
#import tensorflow as tf
#import joblib

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import adfuller,acf, pacf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from pylab import rcParams
from vnstock import *

In [2]:
companyName = 'AAPL'
start_date = "2022-12-01" 
end_date = "2023-12-01" 
df = yf.download(companyName, start_date, end_date)
close = df["Close"].tolist()

[*********************100%%**********************]  1 of 1 completed


In [4]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,251.000000,251.000000,251.000000,251.000000,251.000000,2.510000e+02
mean,167.702869,169.329442,166.332709,167.914303,167.423999,6.142316e+07
std,18.571844,18.399008,18.728641,18.553242,18.709766,1.854627e+07
min,126.010002,127.769997,124.169998,125.019997,124.325081,2.404830e+07
25%,152.460007,154.294998,150.855003,152.930000,152.312157,4.936340e+07
50%,172.619995,174.029999,171.470001,173.000000,172.772385,5.697620e+07
75%,181.150002,182.389999,179.004997,180.955002,180.475220,6.878810e+07
max,196.240005,198.229996,195.279999,196.449997,195.926956,1.601569e+08


In [6]:
df.reset_index(inplace=True)

In [9]:
df = df.rename(columns={'Open': 'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume', 'Date':'date'})

In [10]:
df.head()

,date,open,high,low,close,Adj Close,volume
0,2022-12-01,148.210007,149.130005,146.610001,148.309998,147.485626,71250400
1,2022-12-02,145.960007,148.000000,145.649994,147.809998,146.988403,65447400
2,2022-12-05,147.770004,150.919998,145.770004,146.630005,145.814972,68826400
3,2022-12-06,147.070007,147.300003,141.919998,142.910004,142.115646,64727200
4,2022-12-07,142.190002,143.369995,140.000000,140.940002,140.156601,69721100


In [13]:
# Normalaze Data For Create Trin and Test with new dataframe with only the 'Close' column 
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
# fit scaler on only one column('close')
scaled = scaler.fit_transform(df.filter(['close'])) 
# now have a scaled dataframe with 'colose column'
df2=pd.DataFrame(scaled, columns=['close'])

In [15]:
print(df2.head())
print(df2.describe())

      close
0  0.326053
1  0.319054
2  0.302534
3  0.250455
4  0.222876
            close
count  251.000000
mean     0.600508
std      0.259740
min      0.000000
25%      0.390732
50%      0.671707
75%      0.783074
max      1.000000


In [16]:
# Finding a Length of %90 from the number of rows to Create a train Dataset (using array)
new_dataset_length = int(np.ceil( len(df.filter(['close']).values) * .9 )) 
print('Length of %90 of Dataset is ',new_dataset_length, ' Therefore, the test data length is ',len(df.close) - new_dataset_length)

Length of %90 of Dataset is  226  Therefore, the test data length is  25


In [17]:
# Create the training data set with 20 time steps 
# finding the best time step from MA that 20 was the most optimal

train = scaled[0:int(new_dataset_length), :]

time_step = 20

X_train = []
y_train = []

for i in range(time_step, len(train)):
    X_train.append(train[i-time_step:i, 0])
    y_train.append(train[i, 0])
    if i<= (time_step+1):
        print(X_train)
        print(y_train)
        print()

[array([0.32605349, 0.31905363, 0.30253406, 0.25045509, 0.22287562,
       0.24681502, 0.2399553 , 0.27257467, 0.28629434, 0.25465505,
       0.16071683, 0.13285731, 0.10289792, 0.10191805, 0.14601708,
       0.10093797, 0.09575814, 0.07013863, 0.01427977, 0.06425877])]
[0.06873856939293721]

[array([0.32605349, 0.31905363, 0.30253406, 0.25045509, 0.22287562,
       0.24681502, 0.2399553 , 0.27257467, 0.28629434, 0.25465505,
       0.16071683, 0.13285731, 0.10289792, 0.10191805, 0.14601708,
       0.10093797, 0.09575814, 0.07013863, 0.01427977, 0.06425877]), array([0.31905363, 0.30253406, 0.25045509, 0.22287562, 0.24681502,
       0.2399553 , 0.27257467, 0.28629434, 0.25465505, 0.16071683,
       0.13285731, 0.10289792, 0.10191805, 0.14601708, 0.10093797,
       0.09575814, 0.07013863, 0.01427977, 0.06425877, 0.06873857])]
[0.06873856939293721, 0.0007000287210441325]



In [25]:
class MovingAverageTimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length, moving_average_size):
        self.data = data
        self.sequence_length = sequence_length
        self.moving_average_size = moving_average_size

    def __len__(self):
        return len(self.data) - self.sequence_length - self.moving_average_size + 1

    def __getitem__(self, index):
        sequence_end_idx = index + self.sequence_length
        moving_average_end_idx = sequence_end_idx + self.moving_average_size
        sequence = self.data[index:sequence_end_idx]
        
        # Compute the moving average
        moving_average = torch.mean(self.data[sequence_end_idx:moving_average_end_idx])
        return sequence, moving_average

In [29]:
df_tensor = torch.tensor(df['close'], dtype=torch.float32)

In [26]:
sequence_length = 20 
moving_average_size = 20

In [30]:
dataset = MovingAverageTimeSeriesDataset(df_tensor, sequence_length, moving_average_size)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
for sequence, target in dataloader:
    print(sequence.shape, target.shape)

torch.Size([32, 20]) torch.Size([32])
torch.Size([32, 20]) torch.Size([32])
torch.Size([32, 20]) torch.Size([32])
torch.Size([32, 20]) torch.Size([32])
torch.Size([32, 20]) torch.Size([32])
torch.Size([32, 20]) torch.Size([32])
torch.Size([20, 20]) torch.Size([20])


In [33]:
len(dataset)

212

In [55]:
dataset = df['close']

In [56]:
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size

In [59]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [20]:

class TimeSeriesTransformer(nn.Module):
    def __init__(self, num_features, num_layers, num_heads, dropout):
        super(TimeSeriesTransformer, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=num_features, nhead=num_heads, dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(
            self.encoder_layer, num_layers=num_layers
        )
        self.positional_encoding = self._generate_positional_encoding(num_features)
        self.output_layer = nn.Linear(num_features, 1)

    def _generate_positional_encoding(self, num_features):
        # Generate positional encodings here
        pass

    def forward(self, src):
        src += self.positional_encoding[:src.size(0), :]
        output = self.transformer_encoder(src)
        output = self.output_layer(output)
        return output

In [42]:
# Parameters
num_features = 10  # Number of input features
num_layers = 4     # Number of Transformer layers
num_heads = 2      # Number of heads in the multi-head attention mechanism
dropout = 0.1      # Dropout rate
num_epochs = 10

In [43]:
model = TimeSeriesTransformer(num_features, num_layers, num_heads, dropout)

In [44]:
# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [60]:
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_seq, target_seq = batch
        output = model(input_seq)
        loss = criterion(output, target_seq)
        loss.backward()
        optimizer.step()

ValueError: too many values to unpack (expected 2)

1

In [48]:
for batch in train_loader:
    input_seq, target_seq = batch
    a = input_seq

In [54]:
a.type()

'torch.FloatTensor'